## Query Example - Word Count

Let us see how we can perform word count using Spark SQL. Using word count as an example we will understand how we can come up with the solution using pre-defined functions available.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002480


itv002480

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Predefined Functions").
    master("yarn").
    getOrCreate

username = itv002480
spark = org.apache.spark.sql.SparkSession@11f6afd8


org.apache.spark.sql.SparkSession@11f6afd8

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Create table by name lines.
* Insert data into the table.
* Split lines into array of words.
* Explode array of words from each line into individual records.
* Use group by and get the count. We cannot use `GROUP BY` directly on exploded records and hence we need to use nested sub query.

In [3]:
%%sql

DROP DATABASE IF EXISTS itv002480_demo CASCADE

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

CREATE DATABASE IF NOT EXISTS itv002480_demo

++
||
++
++



In [5]:
%%sql

USE itv002480_demo

++
||
++
++



In [6]:
%%sql

CREATE TABLE lines (s STRING)

++
||
++
++



In [7]:
%%sql

INSERT INTO lines VALUES
  ('Hello World'),
  ('How are you'),
  ('Let us perform the word count'),
  ('The definition of word count is'),
  ('to get the count of each word from this data')

++
||
++
++



In [8]:
%%sql

SELECT * FROM lines

+--------------------+
|                   s|
+--------------------+
|         Hello World|
|         How are you|
|Let us perform th...|
|The definition of...|
|to get the count ...|
+--------------------+



In [9]:
%%sql

SELECT split(s, ' ') AS word_array FROM lines

+--------------------+
|          word_array|
+--------------------+
|      [Hello, World]|
|     [How, are, you]|
|[Let, us, perform...|
|[The, definition,...|
|[to, get, the, co...|
+--------------------+



In [10]:
spark.sql("SHOW functions").show(300, false)

+---------------------------+
|function                   |
+---------------------------+
|!                          |
|%                          |
|&                          |
|*                          |
|+                          |
|-                          |
|/                          |
|<                          |
|<=                         |
|<=>                        |
|=                          |
|==                         |
|>                          |
|>=                         |
|^                          |
|abs                        |
|acos                       |
|add_months                 |
|aggregate                  |
|and                        |
|approx_count_distinct      |
|approx_percentile          |
|array                      |
|array_contains             |
|array_distinct             |
|array_except               |
|array_intersect            |
|array_join                 |
|array_max                  |
|array_min                  |
|array_pos

In [11]:
%%sql

SELECT explode(split(s, ' ')) AS words FROM lines

+-------+
|  words|
+-------+
|  Hello|
|  World|
|    How|
|    are|
|    you|
|    Let|
|     us|
|perform|
|    the|
|   word|
+-------+
only showing top 10 rows



In [12]:
%%sql

SELECT count(1) FROM (SELECT explode(split(s, ' ')) AS words FROM lines)

+--------+
|count(1)|
+--------+
|      27|
+--------+



In [13]:
%%sql

SELECT explode(split(s, ' ')) AS words, count(1) FROM lines
GROUP BY explode(split(s, ' '))

Magic sql failed to execute with error: 
Generators are not supported outside the SELECT clause, but got: 'Aggregate [explode(split(s#51,  ))], [explode(split(s#51,  )) AS words#49, count(1) AS count(1)#52L];

In [14]:
%%sql

SELECT word, count(1) FROM (
  SELECT explode(split(s, ' ')) AS word FROM lines
) q
GROUP BY word

+-----+--------+
| word|count(1)|
+-----+--------+
|World|       1|
|   us|       1|
|  you|       1|
|count|       3|
|   is|       1|
| each|       1|
| data|       1|
|Hello|       1|
|  How|       1|
|  the|       2|
+-----+--------+
only showing top 10 rows



In [15]:
%%sql

SELECT count(1) FROM
(
    SELECT word, count(1) FROM (
        SELECT explode(split(s, ' ')) AS word FROM lines
    ) q
    GROUP BY word
)

+--------+
|count(1)|
+--------+
|      21|
+--------+

